In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [3]:
import re

In [4]:
# from nltk.corpus import stopwords
# stops = set(stopwords.words("english"))

In [5]:
CLUSTER_COUNT = 20

In [6]:
METHOD = "bow_kmeans"
DATA_FOLDER = "data/"
OUTPUT_FOLDER = "output/"
TITLE_FILE = DATA_FOLDER + "title_StackOverflow.txt"
CHECK_INDEX_FILE = DATA_FOLDER + "check_index.csv"

In [7]:
if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

# Read and cleaning data

In [8]:
# # substitute symbols with whitespaces, to lowercase, remove stopwords
# def cleanTitle(title):
#     return ' '.join([w for w in re.sub("[^a-zA-Z]", " ", title).lower().split() if not w in stops])

In [9]:
# substitute symbols with whitespaces, to lowercase
def cleanTitle(title):
    return re.sub("[^a-zA-Z]", " ", title).lower()

In [10]:
with open(TITLE_FILE) as f:
    titles = f.read().splitlines()

In [11]:
clean_titles = [cleanTitle(title) for title in titles]

In [12]:
clean_titles[:5]

['how do i fill a dataset or a datatable from a linq query resultset  ',
 'how do you page a collection with linq ',
 'best subversion clients for windows vista    bit ',
 'best practice  collaborative environment  bin directory  svn',
 'visual studio setup project   per user registry settings']

# Clustering

In [13]:
vectorizer = CountVectorizer(analyzer = "word",   \
                             tokenizer = None,    \
                             preprocessor = None, \
                             stop_words = "english",   \
                             max_features = 5000) 

In [14]:
# vectorizer = TfidfVectorizer(analyzer="word", stop_words='english')

In [15]:
title_vecs = vectorizer.fit_transform(clean_titles)

In [16]:
title_vecs.shape

(20000, 5000)

In [17]:
model = KMeans(n_clusters=CLUSTER_COUNT, init='k-means++', max_iter=100, n_init=1)
model.fit(title_vecs)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=20, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [18]:
title_clusters = np.array(model.labels_.tolist())

# output

In [19]:
checkIndexDF = pd.read_csv(CHECK_INDEX_FILE)

In [20]:
checkIndexDF.head()

,ID,x_ID,y_ID
0,0,11726,1565
1,1,16528,7523
2,2,16683,19401
3,3,6506,5239
4,4,10279,19928


In [21]:
checkIndexDF["Ans"] = (title_clusters[checkIndexDF["x_ID"]] == title_clusters[checkIndexDF["y_ID"]]).astype(int)

In [22]:
checkIndexDF.head(20)

,ID,x_ID,y_ID,Ans
0,0,11726,1565,0
1,1,16528,7523,0
2,2,16683,19401,0
3,3,6506,5239,0
4,4,10279,19928,0
5,5,19846,2428,0
6,6,3532,18102,0
7,7,11502,9753,0
8,8,13470,18420,0
9,9,4867,4793,0


In [23]:
checkIndexDF.to_csv(OUTPUT_FOLDER + METHOD + ".csv", columns=["ID", "Ans"], index=False)